In [1]:
%pip install -q pymupdf4llm aiofiles

Note: you may need to restart the kernel to use updated packages.


In [1]:
import asyncio
import logging
import os
from typing import Dict

import aiofiles
import pymupdf4llm

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger(__name__)

In [2]:
async def office_to_pdf(file_path: str) -> str:
    output_dir = os.path.dirname(file_path)
    base_name = os.path.basename(file_path)
    pdf_file = os.path.join(output_dir, os.path.splitext(base_name)[0] + ".pdf")

    try:
        process = await asyncio.create_subprocess_exec(
            "libreoffice",
            "--headless",
            "--convert-to",
            "pdf",
            file_path,
            "--outdir",
            output_dir,
            stdout=asyncio.subprocess.PIPE,
            stderr=asyncio.subprocess.PIPE,
        )
        stdout, stderr = await process.communicate()

        if process.returncode != 0:
            logger.error(
                f"Error converting {file_path} to PDF: {stderr.decode()}"
            )
            raise RuntimeError(f"Failed to convert {file_path} to PDF")

        logger.info(f"Successfully converted {file_path} to PDF")
        return pdf_file
    except Exception as e:
        logger.error(f"Error in office_to_pdf: {str(e)}")
        raise


async def read_markdown_file(file_path: str) -> str:
    try:
        async with aiofiles.open(file_path, mode="r") as file:
            content = await file.read()
        return content
    except Exception as e:
        logger.error(f"Error reading markdown file {file_path}: {str(e)}")
        raise


async def convert_to_markdown(file_path: str) -> Dict[str, str]:
    _, ext = os.path.splitext(file_path)
    ext = ext.lower()

    metadata = {
        "original_file": file_path,
        "file_type": ext[1:],  # Убираем точку из расширения
        "conversion_method": "",
    }

    converters = {
        (".doc", ".docx", ".rtf"): lambda f: (
            pymupdf4llm.to_markdown(
                f,
                write_images=False,
                embed_images=False,
                graphics_limit=None,
                margins=(0, 0, 0, 0),
                table_strategy="lines_strict",
                fontsize_limit=1,
                ignore_code=True,
                show_progress=False,
            ),
            "office_to_pdf",
        ),
        ".pdf": lambda f: (
            pymupdf4llm.to_markdown(
                f,
                write_images=False,
                embed_images=False,
                graphics_limit=None,
                margins=(0, 0, 0, 0),
                table_strategy="lines_strict",
                fontsize_limit=1,
                ignore_code=True,
                show_progress=False,
            ),
            "direct_pdf",
        ),
        ".md": read_markdown_file,
    }

    try:
        if ext in (".doc", ".docx", ".rtf"):
            pdf_file = await office_to_pdf(file_path)
            content, method = converters[".pdf"](pdf_file)
        elif ext == ".md":
            content = await converters[".md"](file_path)
            method = "direct_markdown"
        elif ext == ".pdf":
            content, method = converters[".pdf"](file_path)
        else:
            raise ValueError(f"Unsupported file extension: {ext}")

        metadata["conversion_method"] = method
        logger.info(f"Successfully converted {file_path} to markdown")
        return {"content": content, "metadata": metadata}
    except Exception as e:
        logger.error(f"Error converting {file_path} to markdown: {str(e)}")
        raise


async def process_file(file_path: str) -> Dict[str, str]:
    try:
        return await convert_to_markdown(file_path)
    except Exception as e:
        logger.error(f"Error processing file {file_path}: {str(e)}")
        raise

In [3]:
# Пример использования
file_path = '../data/public/Б.1 Б.1.2 Деловой иностранный язык.pdf'
markdown_content = await process_file(file_path)

2024-10-08 22:04:15,779 - INFO - Successfully converted ../data/public/Б.1 Б.1.2 Деловой иностранный язык.pdf to markdown


In [4]:
from IPython.display import Markdown

Markdown(markdown_content['content'])

-----

-----

**УТВЕРЖДАЮ**
Заместитель директора по учебной
работе  Гуманитарно-педагогической
академии (филиал) ФГАОУ ВО «КФУ
имени В.И. Вернадского» в г. Ялте
_____________ Т.А. Кот
«____» ___________ 2017 г.

**РАБОЧАЯ ПРОГРАММА ДИСЦИПЛИНЫ**

**ДЕЛОВОЙ ИНОСТРАННЫЙ ЯЗЫК**

Направление подготовки: 09.04.03 Прикладная информатика

Направленность программы: Информационные системы и технологии корпоративного

управления

Квалификация выпускника: магистр

Ялта 2017


-----

Рабочая программа дисциплины «Деловой иностранный язык» для обучающихся по
направлению подготовки 09.04.03. Прикладная информатика, направленность программы
«Информационные системы и технологии корпоративного управления».

Программа составлена в соответствии с требованиями ФГОС ВО № 1404

от 30 октября 2014 г. и учебным планом КФУ для очной и заочной формы обучения
от «__» ____________ 2017 г.

**Разработчик: Осадчая Татьяна Юрьевна, кандидат педагогических наук, доцент кафедры**
иностранной филологии и методики преподавания.

Рабочая программа утверждена на заседании кафедры иностранной филологии и методики
преподавания
Протокол от «28» августа 2017 г. № 1

Заведующий кафедрой
иностранной филологии
и методики преподавания _______________  канд. филол. наук, доцент  А. А. Майборода

**Согласовано:**
с учебно-методической комиссией Института филологии, истории и искусств Гуманитарнопедагогической академии (филиал) ФГАОУ ВО «КФУ им. В.И. Вернадского» в г. Ялте
Протокол от «29» августа 2017 г. № 1

Председатель          ________________  канд. пед. наук, доцент А. А. Береснев

Зав. библиотекой
Гуманитарно-педагогической академии (филиал)
ФГАОУ ВО «КФУ им. В.И. Вернадского» в г. Ялте _____________ Л. А. Ковалерист


-----

**1. Перечень планируемых результатов обучения по дисциплине, соотнесенных с**

**планируемыми результатами освоения ОПОП ВО**

Основной целью курса является совершенствование иноязычной коммуникативной

компетенции выпускников, обеспечение уровня знаний и умений, который позволит
пользоваться иностранным языком в различных областях профессиональной деятельности,
научной и практической работе, в общении с зарубежными партнерами, для
самообразовательных и других целей. Наряду с практической целью, курс реализует
образовательные и воспитательные цели, способствуя расширению кругозора студентов,
повышению их общей культуры и образования, а также культуры мышления и
повседневного и профессионального общения, воспитанию терпимости и уважения к
духовным ценностям народов других стран.

В результате изучения данной дисциплины обучающийся должен обладать

систематизированными базовыми знаниями, умениями и навыками в обозначенной сфере, а
также уметь практически их применить в процессе самостоятельной работы и при написании
научных работ.

**2. Место дисциплины в структуре ОПОП ВО**

Дисциплина «Деловой иностранный язык» входит в базовую часть учебного плана по

направлению подготовки 09.04.03. Прикладная информатика, направленность программы
«Информационные системы и технологии корпоративного управления».

Изучение дисциплины «Деловой иностранный язык» требует наличия у студентов

знаний по курсу «Иностранный язык».

Дисциплина «Деловой иностранный язык» необходима для осуществления научно
исследовательской работы, а также дальнейшей профессиональной деятельности.
Полученные в ходе изучения курса знания, приобретенные умения и навыки могут быть
использованы при написании магистерской работы.

Дисциплина изучается на 1-2 курсе, в 1-4 семестрах.

**Результаты обучения дисциплине, характеризующие этапы**

**формирования компетенций**

Таблица 1

|Коды компетен ций|Результаты освоения основной профессиональной образовательной программы (компетенция или содержание достигнутого уровня освоения компетенции)|Результаты обучения|
|---|---|---|
|ОПК-1|Способность к коммуникации в устной и письменной формах на русском и иностранном языках для решения задач профессиональной деятельности|Знать: алгоритм работы с профессионально- ориентированной и деловой информацией на иностранном языке. Уметь: вести устную и письменную профессиональную коммуникацию на иностранном языке. Владеть: межкультурной коммуникативной компетенцией в профессиональной и деловой сферах деятельности.|


-----

**3. Объем дисциплины в зачетных единицах с указанием академических или**
**астрономических часов, выделенных на контактную работу обучающихся с**

**преподавателем (по видам учебных занятий) и на самостоятельную работу**

**обучающихся**

**Всего часов/ ЗЕ**

очная заочная

**Виды контактной и внеаудиторной работы**

форма форма

обучения обучения

Общий объем дисциплины 216/6 216/6

Аудиторная работа 64 44

в том числе:

Лекции

Лабораторные занятия 64 44

Практические (семинарские) занятия

Самостоятельная работа обучающихся 152 172

Контрольная работа 1,3 сем.

Иные виды работ:

Курсовое проектирование

Курсовая работа

Формы промежуточной аттестации:

Зачет 1,3 сем. 1,3 сем.

Экзамен 2,4 сем. 2,4 сем.

**4. Содержание дисциплины, структурированное по темам (разделам) с указанием**

**отведенного на них количества академических или астрономических часов и видов**

**учебных занятий**

Таблица 4.1

Разделы дисциплины

|обучающихся|Col2|Col3|
|---|---|---|
|Виды контактной и внеаудиторной работы|Всего часов/ ЗЕ||
||очная форма обучения|заочная форма обучения|
|Общий объем дисциплины|216/6|216/6|
|Аудиторная работа|64|44|
|в том числе:|||
|Лекции|||
|Лабораторные занятия|64|44|
|Практические (семинарские) занятия|||
|Самостоятельная работа обучающихся|152|172|
|Контрольная работа||1,3 сем.|
|Иные виды работ:|||
|Курсовое проектирование|||
|Курсовая работа|||
|Формы промежуточной аттестации:|||
|Зачет|1,3 сем.|1,3 сем.|
|Экзамен|2,4 сем.|2,4 сем.|

|№ п/п|Наименование разделов дисциплины|Всего часов, очн. (заочн.)|В том числе очное (заочное)|Col5|Col6|Col7|
|---|---|---|---|---|---|---|
||||ауди- торные занятия|Из них||Самостоя тельная работа|
|||||Лекции|Лаборато рные занятия||
|1.|Раздел 1. Jobs. Careers. Companies.|54|10(8)||10(8)|44(46)|
|2.|Раздел 2. Negotiating. Meetings. Conferences.|54|20(14)||20(14)|34(40)|
|3.|Раздел 3. Formal correspondence. Presentations. Reports.|54|20(8)||20(8)|34(46)|
|4.|Раздел 4. Business cultures across the world.|54|14(14)||14(14)|40(40)|


-----

Таблица 4.2


Темы разделов дисциплины

|Раздел|Номер занятия|Вид занятия|Номер вида занятия|Тема занятия|Часы|
|---|---|---|---|---|---|
|Раздел 1. Jobs. Careers. Companies.|1|Лабораторное занятие|1|Applying for a job. A job interview. Writing a good resume.|2|
||2|Лабораторное занятие|2|A business meeting. Making a strong start of a meeting.|2|
||3|Лабораторное занятие|3|Making telephone talks. Playing a role in team meetings.|2|
||4|Лабораторное занятие|4|Asking questions for factual information. Questioning styles. Business trips.|2|
||5|Лабораторное занятие|5|What is a business company? The company’s structure.|2|
|Раздел 2. Negotiating. Meetings. Conferences.|6-7|Лабораторное занятие|6-7|Negotiating. Making proposals and counter-proposals. Writing an email, fax.|4|
||8-9|Лабораторное занятие|8-9|Chairing a meeting. The functions of a chairperson. Negotiations and contacts.|4|
||10-11|Лабораторное занятие|10-11|Emphasizing your point and your priorities. Writing memos.|4|
||12-13|Лабораторное занятие|12-13|Holding conferences. Dealing with a crisis. Finding the best solution of a problem.|4|
||14-15|Лабораторное занятие|14-15|Presentation styles. Giving presentations. The main characteristics of a market and pricing.|4|
|Раздел 3. Formal correspondence. Presentations. Reports.|16-17|Лабораторное занятие|16-17|Formal correspondence. Company’s finances. Financial statements. Writing a formal letter.|4|
||18-19|Лабораторное занятие|18-19|Reaching agreement. Negotiating styles.|4|
||20-21|Лабораторное занятие|20-21|Selling your idea. Presenting a proposal. Giving presentations.|4|
||22-23|Лабораторное занятие|22-23|Summarizing in meetings. Reported speech. Giving an accurate report.|4|
||24-25|Лабораторное занятие|24-25|Writing minutes of the meeting. Recording decisions and actions. Recording key information.|4|
|Разде л 4.|26-27|Лабораторное занятие|26-27|British business culture.|4|


-----

|Col1|28-29|Лабораторное занятие|28-29|American business culture.|4|
|---|---|---|---|---|---|
||30-31|Лабораторное занятие|30-31|Canadian business culture.|4|
||32|Лабораторное занятие|32|Australian business culture.|2|
||ИТОГО АУДИТОРНЫХ ЧАСОВ ПО ДИСЦИПЛИНЕ||||64|


**5. Перечень учебно-методического обеспечения для самостоятельной работы**

**обучающихся по дисциплине**

1. Агабекян, И. П. Деловой английский = English for business : учебное пособие / И. П.
Агабекян. – 9–е изд., стер. – Ростов н/Д. : Феникс, 2013. – 319 с. : ил. – (Высшее
образование). – Библиогр.: с. 315.
2. Гарагуля, С. И. Английский язык для делового общения = Learning buisness communication
in English : учебное пособие / С. И. Гарагуля. – Ростов н/Д. : Феникс, 2013. – 269 с. : ил. –
(Высшее образование). – Библиогр.: с. 263–265.
3. Гусева, Т.С. Уверенное общение в деловом английском: учебное пособие. Ч. II /
Т.С. Гусева, Г.И. Дедкова. – М. : Владос, 2014. – 200 с.
4. Бердышев, С. Н. Деловой английский. Стандарты, документация, корреспонденция :

[русско–английский толковый словарь] / С. Н. Бердышев. – Ростов н/Д. : Феникс, 2014. – 303
с. : табл. – (Бизнес–класс). – Прил.: с. 293–302.

**6.** **Фонд оценочных средств для проведения промежуточной аттестации**

**обучающихся по дисциплине**

6.1. Результаты обучения по дисциплине, формы промежуточной аттестации и виды
оценочных средств

|Результаты обучения по дисциплине, необходимые для формировани я компетенции или ее части|Критерии и шкала оценки результатов обучения по дисциплине|Формы промеж уточной аттеста ции|Вид оценочн ых средств|
|---|---|---|---|
|Знать: - алгоритм работы с профессиональ но- ориентированн ой и деловой информацией на иностранном языке. Уметь: - вести устную|Оценки «отлично» А (90-100) заслуживает обучающийся, подтвердивший: - всесторонние, систематические и глубокие знания учебного и нормативного материала по изучаемой дисциплине; - умение свободно выполнять задания, предусмотренные программой, и дополнительной учебной литературой, рекомендованной кафедрой; - навыки уверенного и творческого применения полученных знаний и умений при решении профессиональных задач. Оценки «хорошо» В (82-89) заслуживает обучающийся, подтвердивший:|Зачет, экзамен|Устный опрос, Контрол ьная работа, тестиров ание|


-----

|и письменную профессиональ ную коммуникацию на иностранном языке. Владеть: - межкультурной коммуникативн ой компетенцией в профессиональ ной и деловой сферах деятельности.|- полное знание учебного материала, предусмотренного программой; - умение успешно выполнять предусмотренные в программе задания; - навыки уверенного применения полученных знаний и умений при решении профессиональных задач. Оценки «хорошо» С (74-81) заслуживает обучающийся, подтвердивший: - знание учебного материала, предусмотренного программой изучаемой дисциплины; - умение выполнять предусмотренные в программе задания; - навыки применения полученных знаний и умений при решении профессиональных задач. Оценки «удовлетворительно» D (64-73) заслуживает обучающийся, подтвердивший: - знание основного учебного материала в достаточном объеме; - умение справляться с выполнением заданий, предусмотренных программой; - навыки применения полученных знаний и умений при работе с профессиональными задачами. Оценки «удовлетворительно» Е (60-63) заслуживает обучающийся, подтвердивший: - знание основного учебного материала в объеме, необходимом для дальнейшего обучения; - умение частично справляться с выполнением заданий, предусмотренных программой; - навыки фрагментарного применения полученных знаний и умений при работе с профессиональными задачами. Оценки «неудовлетворительно» FX (35-59) заслуживает обучающийся, подтвердивший знания основного учебного материала на поверхностном уровне; не справившийся с выполнением заданий, предусмотренных программой. Оценки «неудовлетворительно» F (1-34) заслуживает обучающийся, ответы которого могут быть оценены ниже требований, сформулированных в предыдущем пункте. Необходим повторный курс учебной дисциплины.|Col3|Col4|
|---|---|---|---|


**6.2. Формы текущего контроля**

**и виды используемых оценочных средств по дисциплине**


-----

|Разделы дисциплин ы|Результаты изучения разделов дисциплин (знать, уметь, владеть)|Критерии и шкала оценки результатов изучения разделов дисциплины|Вид текущего контроля|Вид оценочных средств|
|---|---|---|---|---|
|Раздел 1. Jobs. Careers. Companies.|Знать: - алгоритм работы с профессионально- ориентированной и деловой информацией на иностранном языке. Уметь: - вести устную и письменную профессиональную коммуникацию на иностранном языке. Владеть: - межкультурной коммуникативной компетенцией в профессиональной и деловой сферах деятельности.|Критерии оценки результатов: степень достижения поставленных для первого раздела задач (знать, уметь, владеть) Шкала оценки результатов: см. пункт 6.3, табл. «Критерии и шкала оценивания»|Устный опрос, Контроль ная работа, тестирова ние|Набор вопросов и заданий по каждой теме Карточки для тестирования Бланки для написания деловой документации Карточки для ролевых игр по деловой тематике|
|Раздел 2. Negotiating. Meetings. Conferences .|Знать: - алгоритм работы с профессионально- ориентированной и деловой информацией на иностранном языке. Уметь: - вести устную и письменную профессиональную коммуникацию на иностранном языке. Владеть: - межкультурной коммуникативной компетенцией в профессиональной|Критерии оценки результатов: степень достижения поставленных для второго раздела задач (знать, уметь, владеть) Шкала оценки результатов: см. пункт 6.3, табл. «Критерии и шкала оценивания»|Устный опрос, Контроль ная работа, тестирова ние|Набор вопросов и заданий по каждой теме Карточки для тестирования Бланки для написания деловой документации Карточки для ролевых игр по деловой тематике|


-----

|Col1|и деловой сферах деятельности.|Col3|Col4|Col5|
|---|---|---|---|---|
|Раздел 3. Formal corresponde nce. Presentation s. Reports.|Знать: - алгоритм работы с профессионально- ориентированной и деловой информацией на иностранном языке. Уметь: - вести устную и письменную профессиональную коммуникацию на иностранном языке. Владеть: - - межкультурной коммуникативной компетенцией в профессиональной и деловой сферах деятельности.|Критерии оценки результатов: степень достижения поставленных для третьего раздела задач (знать, уметь, владеть) Шкала оценки результатов: см. пункт 6.3, табл. «Критерии и шкала оценивания»|Устный опрос, Контроль ная работа, тестирова ние|Набор вопросов и заданий по каждой теме Карточки для тестирования Бланки для написания деловой документации Карточки для ролевых игр по деловой тематике|
|Раздел 4. Business cultures across the world.|Знать: - алгоритм работы с профессионально- ориентированной и деловой информацией на иностранном языке. Уметь: - вести устную и письменную профессиональную коммуникацию на иностранном языке. Владеть: - межкультурной коммуникативной компетенцией в профессиональной и деловой сферах деятельности.|Критерии оценки результатов: степень достижения поставленных для четвертого раздела задач (знать, уметь, владеть) Шкала оценки результатов: см. пункт 6.3, табл. «Критерии и шкала оценивания»|Устный опрос, Контроль ная работа, тестирова ние|Набор вопросов и заданий по каждой теме Карточки для тестирования Бланки для написания деловой документации Карточки для ролевых игр по деловой тематике|


-----

**6.3. Методические материалы, определяющие процедуры оценивания знаний, умений,**
**навыков и (или) опыта деятельности, характеризующие этапы формирования**
**компетенций**

Общая оценка выставляется с учетом результатов всех этапов текущего контроля в

порядке, установленном Положением о балльно-рейтинговой системе и Положением о
промежуточной аттестации образовательной организации. Текущий контроль по

дисциплине, является тематическим, и процедурно состоит из нескольких этапов:

1) контроль освоения теоретического материала по каждой теме курса, проводится в

форме устного опроса (собеседование), тестов (приложение); контрольной работы.

2) контроль умений применения полученных знаний, проводится в форме собеседования

по выполнению дополнительных заданий по каждой теме курса, заслушивания и
обсуждения стилистических анализов текстов, выполненных обучающимися в рамках
тематики дисциплины;

3) контроль полученных навыков осуществляется в форме выполнения практических

заданий.

Текущий контроль проводится в конце разделов курса в форме контрольной работы.
Обучающиеся, не получившие положительную оценку по результатам текущего

контроля знаний, умений и навыков, формируемых в процессе изучения дисциплины, не
допускаются к основному этапу - промежуточному контролю в установленном порядке.

Промежуточный контроль осуществляется в форме зачетов и экзаменов, которые

проводятся в устной форме (собеседование).

Общая оценка выставляется с учетом результатов всех этапов контроля в порядке,

установленном кафедрой.

Критерии и шкала оценивания

|Стобалль ная шкала оцениван ия|Шкала оценива ния ECTS|Определение|Национальная пятибалльная шкала оценивания|Col5|
|---|---|---|---|---|
||||экзамен|зачет|
|90-100|А|ОТЛИЧНО - отличное выполнение с незначительным количеством ошибок|5,0 (отлично)|зачтено|
|82-89|В|ОЧЕНЬ ХОРОШО - выше среднего уровня с несколькими ошибками|4,0 (хорошо)||
|74-81|С|ХОРОШО - в общем правильная работа с определенным количеством ошибок|||
|64-73|D|УДОВЛЕТВОРИТЕЛЬНО - неплохо, но с большим количеством недостатков|3,0 (достаточно)||
|60-63|Е|ДОСТАТОЧНО - выполнение удовлетворяет минимальным критериям|||
|35-59|FX|НЕУДОВЛЕТВОРИТЕЛЬНО - возможна повторная сдача|2,0 (неудовлетвори тельно с возможностью повторной сдачи)|не зачтено (с возможностью повторной сдачи)|
|1-34|F|НЕУДОВЛЕТВОРИТЕЛЬНО - необходим повторный курс по учебной дисциплине|2,0 (неудовлетвори тельно с повторным|не зачтено (с повторным изучением дисциплины)|


-----

|Col1|Col2|Col3|изучением дисциплины)|Col5|
|---|---|---|---|---|


**7. Перечень основной и дополнительной учебной литературы, необходимой для**

**освоения дисциплины**

**_а) основная учебная литература:_**
1. Агабекян, И. П. Деловой английский = English for business : учебное пособие / И. П.
Агабекян. – 9–е изд., стер. – Ростов н/Д. : Феникс, 2013. – 319 с. : ил. – (Высшее
образование). – Библиогр.: с. 315.
2. Гарагуля, С. И. Английский язык для делового общения = Learning buisness communication
in English : учебное пособие / С. И. Гарагуля. – Ростов н/Д. : Феникс, 2013. – 269 с. : ил. –
(Высшее образование). – Библиогр.: с. 263–265.
3. Гусева, Т.С. Уверенное общение в деловом английском: учебное пособие. Ч. II /
Т.С. Гусева, Г.И. Дедкова. – М. : Владос, 2014. – 200 с.
4. Колесникова, Н. А. Деловое общение в тестах и интервью : учебное пособие / Н. А.
Колесникова, Л. А. Томашевская. – М. : Флинта ; М. : Наука, 2014. – 212 с. : табл.
5. Маньковская, З. В. Идиомы и фразовые глаголы в деловом общении (английский язык)

[Электронный ресурс] : учебное пособие / З.В. Маньковская. – М.: ИНФРА–М, 2011. – 184 с.:
[60x90 1/16. – (Высшее образование). – Режим доступа: http://znanium.com/catalog.php?](http://znanium.com/catalog.php?bookinfo=205580)
[bookinfo=205580](http://znanium.com/catalog.php?bookinfo=205580)
6. Попов, Е. Б. Деловой английский язык [Электронный ресурс] / Е.Б. Попов – М. : НИЦ
ИНФРА–М, 2015. – 64 с. – Режим доступа: http://znanium.com/catalog.php?bookinfo=515334
**_б) дополнительная учебная литература:_**

1. Бердышев, С. Н. Деловой английский. Стандарты, документация, корреспонденция :

[русско–английский толковый словарь] / С. Н. Бердышев. – Ростов н/Д. : Феникс, 2014. – 303
с. : табл. – (Бизнес–класс). – Прил.: с. 293–302.

2. Cotton D. Market Leader: course Book: Pre-Intermediate: Business English / David Cotton,
David Falvey, Simon Kent. - USA : Financial Times, 2012. - 160 p.

3. Johnson Christine, Barrall Irene. Intelligent business: skills book / C. Johnson, I. Barrall. –
Harlow : Pearson, 2013. – 112 p.

**8. Перечень ресурсов информационно-телекоммуникационной сети «интернет»,**

**необходимых для освоения дисциплины**

[1. http :// learnamericanenglishonline . com](http://learnamericanenglishonline.com/)

[2. http :// learnenglish . britishcouncil . org / en](http://learnenglish.britishcouncil.org/en)

3. http://lingust.ru/english/business – Сайт в помощь изучающим деловой английский язык
4. http://www.bbc.co.uk/news/business/ – Деловые новости BBC
[5. http :// www . bbc . uk / podcast](http://www.bbc.uk/podcast)

[6. http :// www . elllo . org](http://www.elllo.org/)

[7. http :// www . englishprofile . org](http://www.englishprofile.org/)

8. http://www.forbes.com/business/ – Деловые новости Forbes
[9. http :// www .languagelab](http://www/)

[10. http :// www . linguanet . ru](http://www.linguanet.ru/)

11. http://www.lingvo.ua/
12. http://www.market-leader.net/ – Сайт с дополнительными учебно-методическими

материалами к учебникам серии Market Leader.

13. http://www.multitran.ru/
14. http://www.native-english.ru/
15. http://www.rbc.ru/ – Деловые новости РосБизнесКонсалтинг
[16. http://www.sciencedaily.com/](http://www.sciencedaily.com/)

[17. http :// www . study . ru /](http://www.study.ru/)


-----

**9. Методические указания**

**для обучающихся по освоению дисциплины**

|Вид учебных занятий в соответствии с пунктом 4|Методические указания для обучающихся по освоению дисциплины по видам учебных занятий|
|---|---|
|Лабораторн ые занятия|На лабораторных занятиях студенты учатся: - активно использовать в устной (подготовленной и неподготовленной) и письменной речи изученные слова, словосочетания, речевые образцы в пределах тем, предусмотренных программой, в том числе при переводе с английского на русский и с русского на английский; - работать над разными видами аутентичных текстов (главным образом, официально-деловых), что способствует развитию мышления студентов, их иноязычно-речевой способности, лингвострановедческой компетенции, различных видов памяти, воображения, общеречевых и общеучебных умений, а также умению самостоятельной работы с языком. Содержание лабораторного занятия определяется его формой и зависит от тематики. Это, главным образом, лексико-грамматический анализ прочитанного или прослушанного текста, выполнение практических заданий (языковых упражнений, направленных на закрепление вокабуляра, условно-речевых и рецептивных упражнений, направленных на дальнейшую автоматизацию и использование отрабатываемого вокабуляра в письменной и устной, в том числе, спонтанной, речи), беседа по теме. Для подготовки к лабораторным занятиям необходимо пользоваться словарем, прорабатывать и переводить аутентичные тексты, использовать учебные пособия и практикумы; рекомендуется составлять глоссарий, изучать электронные материалы на интернет-сайтах. При выполнении упражнений и переводе текстов рекомендуется делать это в письменной форме для более эффективного закрепления материала, при изучении лексики студентам желательно составлять схемы, таблицы. Учебно-методическое обеспечение: Агабекян, И. П. Деловой английский = English for business : учебное пособие / И. П. Агабекян. – 9–е изд., стер. – Ростов н/Д. : Феникс, 2013. – 319 с. : ил. – (Высшее образование). – Библиогр.: с. 315. 2. Гарагуля, С. И. Английский язык для делового общения = Learning buisness communication in English : учебное пособие / С. И. Гарагуля. – Ростов н/Д. : Феникс, 2013. – 269 с. : ил. – (Высшее образование). – Библиогр.: с. 263–265. 3. Гусева, Т.С. Уверенное общение в деловом английском: учебное пособие. Ч. II / Т.С. Гусева, Г.И. Дедкова. – М. : Владос, 2014. – 200 с. 4. Колесникова, Н. А. Деловое общение в тестах и интервью : учебное пособие / Н. А. Колесникова, Л. А. Томашевская. – М. : Флинта ; М. : Наука, 2014. – 212 с. : табл.|
|Контрольная работа|Контрольная работа по разделам курса проводится в письменной форме. Содержание предусматривает ответы на теоретические вопросы и выполнение практических заданий. При подготовке к контрольной работе обучающимся необходимо повторить материал по разделам дисциплины. Учебно-методическое обеспечение: 1. Попов, Е. Б. Деловой английский язык [Электронный ресурс] / Е.Б. Попов – М. : НИЦ ИНФРА–М, 2015. – 64 с. – Режим доступа:|


-----

|Col1|http://znanium.com/catalog.php?bookinfo=515334|
|---|---|


**10. Перечень информационных технологий, используемых при осуществлении**

**образовательного процесса по дисциплине, включая перечень программного**

**обеспечения и информационных справочных систем (при необходимости)**

 ресурсы глобальной сети Интернет

 пакет прикладных программ LibreOffice (модули «Презентация», «Текстовый

документ» и др.

**11. Описание материально-технической базы, необходимой для осуществления**

**образовательного процесса по дисциплине**

|№ п/п|Наименование оборудованных учебных кабинетов, лабораторий|Перечень оборудования и технических средств обучения|
|---|---|---|
|1.|Компьютерная аудитория|Персональные компьютеры с доступом в Интернет|


-----

**Приложение**

**Оценивание уровня сформированности компетенций**

**Пороговый  уровень**

1. You have sent a letter of application to a college, together with your curriculum vitae which
the college requested. What do you say in the letter to explain that your curriculum vitae is
attached?
_a. You asked for my curriculum vitae, so here it is._
_b. As you can see, I've enclosed my curriculum vitae._
_c. As you requested, I enclose my curriculum vitae._
2. You are writing a letter to the head teacher of a school or college, but you don't know their
name. How do you begin your letter?
_a. Dear head teacher._
_b. Dear Sir / Madam._
_c. Dear Sir._
3. You have received a letter from the manager of a company which buys computer components
from your company, and you are now replying. What do you say?
_a. Thank you for your letter._
_b. Thanks a lot for your letter._
_c. It was great to hear from you._
4. You recently stayed in a hotel and were very unhappy with the service you received. You are
now writing to the manager. What do you say?
_a. I had a horrible time at your hotel recently._
_b. I would like to say that I am unhappy about your hotel._
_c. I would like to complain about the service I received at your hotel recently._

**Базовый  уровень**

**Work in pairs. Make up a dialog using the following information. Role-play your dialog.**
**Student 1**
Your company: MTS – a mobile phone network operator, 3 500 employees, based in Sydney,
Australia, operates in 16 countries, main competitors are Vodafone.
You meet student 2 at a conference. You don’t know each other.

  - Introduce yourself

  - Find out the other’s person name, job, and company

  - Tell them what your company does and some information about it

  - Ask why they are at the conference

  - Ask if you can take their name and telephone number

**Student 2**
Your company: Sweet and Savoury – provides food and drink services, specializes in providing
to film companies, 250 employees, based in Leeds, UK, operates in the UK and Ireland, main
competitors are DirectFood.
You meet student 1 at a conference. You don’t know each other.

  - Introduce yourself

  - Find out the other’s person name, job, and company

  - Tell them what your company does and some information about it

  - Ask why they are at the conference

  - Ask if they can give you a business card


-----

**Продвинутый  уровень**

**Speak on the following:**

1. Imagine that you are attending a conference where you don’t know anyone. It is the first

coffee break and lots of people standing around talking. How would you start a conversation?

2. What are the main parts of a good resume?
3. Which information about the company should you find out before a job interview?
4. You are attending a welcome party at a conference. Start a small talk. Start a conversation

with a least three different people.

5. Which phrases would you use at the beginning of a business meeting?
6. What are the main parts of a business meeting?
7. Prepare a short introduction to your presentation; it should take about 1 minute.
8. What phrases would you use making a business telephone call?
9. What are main roles played at a business meeting?
10. What are the main strategies of stimulating interest during your presentation?
11. What questions would you use when asking for actual information?
12. What questioning styles do you know?
13. Hold an initial meeting of a newly-formed international team of managers.
14. What phrases would you use presenting facts and figures?
15. What are some appropriate topics of an informal talk before a business meeting?
16. Choose one of these roles: coordinator, team-worker, action role, ideas role. Think of how

you might make a contribution at a meeting.

17. Describe a structure of a standard business company.
18. What are the main parts of a report?
19. Which phrases would you use in your email to colleagues to make them more friendly and

diplomatic? Write 2 paragraphs of the email: beginning and ending.

20. What phrases would you use making proposals and counter-proposals?

17


-----

